In [ ]:
#!pip install timm

In [ ]:
#!pip install notebook

In [1]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
!echo $CUDA_HOME

/usr/local/cuda


In [2]:
import numpy as np
import cv2
from PIL import Image
import scipy.io as scio
from pytorch_nndct.apis import torch_quantizer, dump_xmodel
import torch
from torch import nn

import urllib.request

import os
import argparse
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms

import matplotlib.pyplot as plt


[VAIQ_NOTE]: Loading NNDCT kernels...


In [3]:
IMG_W = 256
IMG_H = 192
BATCH_SIZE = 32
BACKBONE = 'resnet18'
TEST_BATCH_SIZE = 1000
SEED = 1
LOG_INTERVAL = 10
RESUME = ''
DATA_ROOT = './build/data'
QUANT_DIR = './build/quantized'
QUANT_MODE = 'float'
DEVICE = 'gpu' 
DEPLOY = False


In [4]:
if torch.cuda.is_available() :
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Testing on {device} device.")

Testing on cuda device.


In [5]:
def replace_relu6_with_hardtanh(model):
    def replace_recursive(module):
        for name, child_module in module.named_children():
            if isinstance(child_module, nn.ReLU6):
                setattr(module, name, nn.Hardtanh(min_val=0, max_val=6))
            else:
                replace_recursive(child_module)

    replace_recursive(model)
    return model


In [6]:
from smallmidas import MidasNet_small
model = MidasNet_small(align_corners=False)
parameters = torch.load('weights/midas_v21_small_256.pt', 
                        map_location=torch.device('cpu'))

if "optimizer" in parameters:
    parameters = parameters["model"]

model.load_state_dict(parameters)
_ = model(torch.randn((1,3,IMG_H,IMG_W)))
replace_relu6_with_hardtanh(model)
model = model.eval()

Loading weights:  None


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite3-b733e338.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_lite3-b733e338.pth

In [7]:
model.to(device)

MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(
        3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        (pad): ZeroPad2d((0, 1, 0, 1))
      )
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=6)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): Hardtanh(min_val=0, max_val=6)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, ker

In [ ]:
"""def forward_hook(module, input, output):
    print(f'{module.__class__.__name__} output shape:', output.shape)

# Register the forward hook to each layer
for name, layer in model.named_modules():
    layer.register_forward_hook(forward_hook)"""

In [8]:
layers = [  model.pretrained.layer1[0],
            model.pretrained.layer1[4][0].conv_dw,
            model.pretrained.layer2[0][0].conv_dw,
            model.pretrained.layer3[0][0].conv_dw,
            model.pretrained.layer4[0][0].conv_dw
         ]

newlayers = []

for custom_conv in layers:
    p = custom_conv.pad.padding
    #p = 1 if min(p[:2]) == 0 else 2
    p = (0,1,0,1) if min(p[:2]) == 0 else (1,2,1,2)
    new_conv_layer = nn.Sequential(
                        nn.ZeroPad2d(p),
                        nn.Conv2d( in_channels=custom_conv.in_channels, 
                                out_channels=custom_conv.out_channels, 
                                kernel_size=custom_conv.kernel_size,
                                stride=custom_conv.stride,
                                #padding = p,
                                bias = custom_conv.bias,
                                dilation = custom_conv.dilation,
                                groups = custom_conv.groups)
    )
                            
    new_conv_layer[1].weight.data.copy_(custom_conv.weight.data)
                            
    newlayers.append(new_conv_layer)

model.pretrained.layer1[0] = newlayers[0]
model.pretrained.layer1[4][0].conv_dw = newlayers[1]
model.pretrained.layer2[0][0].conv_dw = newlayers[2]
model.pretrained.layer3[0][0].conv_dw = newlayers[3]
model.pretrained.layer4[0][0].conv_dw = newlayers[4]

In [9]:
transform = transforms.Compose([
    transforms.Resize((192, 256)),  # Resize to w=256, h=256
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])
from PIL import Image
img = transform(Image.open('/workspace/demodel/dog.jpg')).unsqueeze(0).to(device)

In [10]:
model = model.to(device)
o = model(img).squeeze().detach().cpu().numpy()

min_val = np.min(o)
max_val = np.max(o)
normalized_image = ((o - min_val) / (max_val - min_val)) * 255
normalized_image = normalized_image.astype(np.uint8)
plt.imsave('sonucc.png', normalized_image, cmap='gray')

In [11]:
QUANT_MODE = 'calib'
#target = "0x101000016010407" #4096
target = "0x101000016010406" #3136

quantizer = torch_quantizer(QUANT_MODE, model, (img), output_dir = QUANT_DIR, device=device, 
                            target=target)
qmodel = quantizer.quant_model
oo = qmodel(img)
quantizer.export_quant_config()
quantizer.export_xmodel(QUANT_DIR, deploy_check=True)
quantizer.export_torch_script(output_dir=QUANT_DIR)
quantizer.export_onnx_model(output_dir=QUANT_DIR)


[VAIQ_NOTE]: OS and CPU information:
               system --- Linux
                 node --- yasin
              release --- 6.5.0-41-generic
              version --- #41~22.04.2-Ubuntu SMP PREEMPT_DYNAMIC Mon Jun  3 11:32:55 UTC 2
              machine --- x86_64
            processor --- x86_64

[VAIQ_NOTE]: Tools version information:
                  GCC --- GCC 7.5.0
               python --- 3.8.6
              pytorch --- 1.13.1+cu117
        vai_q_pytorch --- 3.5.0+60df3f1+torch1.13.1+cu117

[VAIQ_NOTE]: GPU information:
          device name --- NVIDIA GeForce GTX 1060
     device available --- True
         device count --- 1
       current device --- 0

[VAIQ_NOTE]: Quant config file is empty, use default quant configuration

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cuda'.

[VAIQ_NOTE]: =>Parsing MidasNet_small...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_MidasNet_small_ed 

██████████████████████████████████████████████████| 273/273 [00:00<00:00, 301.84it/s, OpInfo: name = return_0, type = Return]                                                                                                                                               



[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(./build/quantized/MidasNet_small.py)

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[0]/ZeroPad2d[0]/ret.3) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[4]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.27) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.77) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.127) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer4]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.289) can't be fused

[

I20240711 10:23:53.600904  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:23:53.600944  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:23:53.600977  2223 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B3136_0101000016010406
I20240711 10:23:53.601097  2223 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_conv2d_nndct_relu6_PFLSVe18tEfohRQv, with op num: 9
I20240711 10:23:53.601121  2223 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240711 10:23:53.616398  2223 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240711 10:23:53.616458  2223 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240711 10:23:53.622754  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:23:53.622788  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:23:53.6


[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[1]/Conv2d[conv_pw]/ret.89, op type:nndct_conv2d, output shape: [1, 24, 32, 288]
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[1]/Hardtanh[act1]/ret.93, op type:nndct_relu6, output shape: [1, 24, 32, 288]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[1]/Conv2d[conv_pw]/ret.139, op type:nndct_conv2d, output shape: [1, 12, 16, 576]
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[1]/Hardtanh[act1]/ret.143, op type:nndct_relu6, output shape: [1, 12, 16, 576]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[

...


[VAIQ_NOTE]: =>Exporting quant config.(./build/quantized/quant_info.json)


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/pytorch_nndct/nn/modules/prim_ops.py:116: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if not (list(self.node.out_tensors[0].shape[1:]) == list(input.size())[1:]):
/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/pytorch_nndct/quantization/torchquantizer.py:138: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if NndctOption.nndct_quant_off.value or res.data.numel() == 0:
/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/pytorch_nndct/quantizati


[VAIQ_NOTE]: MidasNet_small_int.pt is generated.(./build/quantized/MidasNet_small_int.pt)


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.8/site-packages/torch/onnx/utils.py:687: UserWarning: Constant folding - Only steps=1 can 


[VAIQ_NOTE]: MidasNet_small_int.onnx is generated.(./build/quantized/MidasNet_small_int.onnx)


In [12]:
QUANT_MODE = 'test'
#target = "DPUCZDX8G_ISA1_B4096" #"0x101000016010407"
target = "DPUCZDX8G_ISA1_B3136" #"0x101000016010406"
quantizer = torch_quantizer(QUANT_MODE, model, (img,), output_dir = QUANT_DIR, device=device,target=target)
qmodel = quantizer.quant_model
oo = qmodel(img)
#quantizer.export_quant_config()
quantizer.export_xmodel(QUANT_DIR, deploy_check=False)
quantizer.export_torch_script(output_dir=QUANT_DIR)
quantizer.export_onnx_model(output_dir=QUANT_DIR)


[VAIQ_NOTE]: OS and CPU information:
               system --- Linux
                 node --- yasin
              release --- 6.5.0-41-generic
              version --- #41~22.04.2-Ubuntu SMP PREEMPT_DYNAMIC Mon Jun  3 11:32:55 UTC 2
              machine --- x86_64
            processor --- x86_64

[VAIQ_NOTE]: Tools version information:
                  GCC --- GCC 7.5.0
               python --- 3.8.6
              pytorch --- 1.13.1+cu117
        vai_q_pytorch --- 3.5.0+60df3f1+torch1.13.1+cu117

[VAIQ_NOTE]: GPU information:
          device name --- NVIDIA GeForce GTX 1060
     device available --- True
         device count --- 1
       current device --- 0

[VAIQ_NOTE]: Quant config file is empty, use default quant configuration

[VAIQ_NOTE]: Quantization test process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cuda'.

[VAIQ_NOTE]: =>Parsing MidasNet_small...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_MidasNet_small_ed is torc

██████████████████████████████████████████████████| 273/273 [00:00<00:00, 278.98it/s, OpInfo: name = return_0, type = Return]                                                                                                                                               



[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(./build/quantized/MidasNet_small.py)

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[0]/ZeroPad2d[0]/ret.3) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[4]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.27) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.77) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.127) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer4]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.289) can't be fused

[

I20240711 10:26:55.988399  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:55.988447  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:55.988462  2223 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B3136
I20240711 10:26:55.988582  2223 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_conv2d_nndct_relu6_jEn2Pyw6T4QgSfxq, with op num: 9
I20240711 10:26:55.988597  2223 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240711 10:26:55.998363  2223 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240711 10:26:55.998425  2223 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240711 10:26:56.003937  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:56.003969  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:56.003983  2223 compi


[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[1]/Conv2d[conv_pw]/ret.89, op type:nndct_conv2d, output shape: [1, 24, 32, 288]
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[1]/Hardtanh[act1]/ret.93, op type:nndct_relu6, output shape: [1, 24, 32, 288]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[1]/Conv2d[conv_pw]/ret.139, op type:nndct_conv2d, output shape: [1, 12, 16, 576]
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[1]/Hardtanh[act1]/ret.143, op type:nndct_relu6, output shape: [1, 12, 16, 576]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[

...


[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[1]/InvertedResidual[1]/19147, op type:nndct_elemwise_add, output shape: [1, 12, 16, 136]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[1]/18689, op type:nndct_elemwise_add, output shape: [1, 12, 16, 96]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/FeatureFusionBlock_custom[scratch]/FeatureFusionBlock_custom[refinenet2]/ResidualConvUnit_custom[resConfUnit2]/ret.461, op type:nndct_elemwise_add, output shape: [1, 24, 32, 128]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[4]/InvertedResidual[2]/18215, op type:nndct_elemwise_add, output shape: [1, 48, 64, 32]



[VAIQ_NOTE]: Find subgraph for e

I20240711 10:26:59.637008  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:59.637039  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:59.637051  2223 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B3136
I20240711 10:26:59.637145  2223 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_elemwise_add_oZS8lfTmi1IEDa6u, with op num: 6
I20240711 10:26:59.637159  2223 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240711 10:26:59.646580  2223 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20240711 10:26:59.646641  2223 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240711 10:26:59.651708  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:59.651742  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:59.651768  2223 compile_pas


[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer4]/Sequential[0]/InvertedResidual[3]/19798, op type:nndct_elemwise_add, output shape: [1, 6, 8, 232]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[1]/InvertedResidual[2]/19239, op type:nndct_elemwise_add, output shape: [1, 12, 16, 136]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/FeatureFusionBlock_custom[scratch]/FeatureFusionBlock_custom[refinenet1]/ResidualConvUnit_custom[resConfUnit2]/ret.485, op type:nndct_elemwise_add, output shape: [1, 48, 64, 64]



[VAIQ_NOTE]: Find subgraph for eltwise_fix_24:
node name:MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[1]/18406, op type:nndct_elemwise_add, output shape: [1, 24, 32, 48]



[VAIQ_NOTE]: Find subgraph for elt

I20240711 10:26:59.851703  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:59.851739  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:59.851752  2223 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B3136
I20240711 10:26:59.851862  2223 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_elemwise_add_rMRLwxDOyHz1bZ9U, with op num: 6
I20240711 10:26:59.851879  2223 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240711 10:26:59.863524  2223 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20240711 10:26:59.863618  2223 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240711 10:26:59.872742  2223 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240711 10:26:59.872809  2223 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240711 10:26:59.872834  2223 compile_pas


[VAIQ_NOTE]: =>Get module with quantization.

[VAIQ_NOTE]: =>Converting to xmodel ...

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[0]/ZeroPad2d[0]/ret.3) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer1]/Sequential[4]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.27) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer2]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.77) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer3]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.127) can't be fused

[VAIQ_WARN]: Pad(MidasNet_small::MidasNet_small/Sequential[pretrained]/Sequential[layer4]/Sequential[0]/InvertedResidual[0]/Sequential[conv_dw]/ZeroPad2d[0]/ret.289) can't be fused

[VAIQ_NOTE]: =>Successfully convert 'MidasNet_

In [13]:
!vai_c_xir -x /workspace/demodel/build/quantized/MidasNet_small_int.xmodel -a /workspace/arch.json -o /workspace/demodel/build/quantized/hereoutput -n MidasNet_small_int

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: null
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B3136_0101000016010406
[UNILOG][INFO] Graph name: MidasNet_small, with op num: 725
[UNILOG][INFO] Begin to compile...
[UNILOG][WARNING] xir::Op{name = MidasNet_small__MidasNet_small_Conv2d_scratch__Conv2d_layer1_rn__ret_397_recomputation_0, type = conv2d} is inserted to do the recomputation for xir::Op{name = MidasNet_small__MidasNet_small_Conv2d_scratch__Conv2d_layer1_rn__ret_397, type = conv2d}.
[UNILOG][WARNING] xir::Op{name = MidasNet_small__MidasNet_small_Conv2d_scratch__Conv2d_layer2_rn__ret_399_recomputation_0, type = conv2d} is inserted to do the recomputation for xir::Op{name = MidasNet_small__MidasNet_small_Conv2d_scratch__Conv2d_layer2_rn__ret_399, type = conv2d}.
[UNILOG][WARNING] xir::Op{name = MidasNet_small__MidasNet_sma